In [2]:
import requests
import pandas as pd

In [30]:
cards = pd.read_csv('commander_cards_all_v1.csv')

## Data preprocessing & cleaning

In [32]:
card_dict = cards.drop(columns=['double-sided'], axis=1) # we don't care if the cards are double-sided, for now

In [33]:
card_dict['card_name'] = card_dict['card_name'].str.split('/|\(', expand=True)[0] # remove all characters after ( or / etc

In [34]:
display(card_dict)

,img_url,card_name,cost
0,https://cards.scryfall.io/large/front/9/f/9f76...,"Eruth, Tormented Prophet",3
1,https://cards.scryfall.io/large/front/f/6/f6cd...,"Esika, God of the Tree",3
2,https://cards.scryfall.io/large/front/8/6/8695...,"Esior, Wardwing Familiar",2
3,https://cards.scryfall.io/large/front/2/e/2e39...,"Esix, Fractal Bloom",6
4,https://cards.scryfall.io/large/front/3/3/3340...,"Estrid, the Masked",4
...,...,...,...
1155,https://cards.scryfall.io/large/front/e/a/ea1a...,"Zaffai, Thunder Conductor",2
1156,https://cards.scryfall.io/large/front/a/c/ac81...,"Zagras, Thief of Heartbeats",2
1157,https://cards.scryfall.io/large/front/8/8/880f...,"Zahid, Djinn of the Lamp",2
1158,https://cards.scryfall.io/large/front/9/c/9c6d...,"Zalto, Fire Giant Duke",2


In [35]:
card_dict['card_name'] = card_dict['card_name'].str.replace('[, ]', '_', regex=True)

In [36]:
display(card_dict)

,img_url,card_name,cost
0,https://cards.scryfall.io/large/front/9/f/9f76...,Eruth__Tormented_Prophet_,3
1,https://cards.scryfall.io/large/front/f/6/f6cd...,Esika__God_of_the_Tree_,3
2,https://cards.scryfall.io/large/front/8/6/8695...,Esior__Wardwing_Familiar_,2
3,https://cards.scryfall.io/large/front/2/e/2e39...,Esix__Fractal_Bloom_,6
4,https://cards.scryfall.io/large/front/3/3/3340...,Estrid__the_Masked_,4
...,...,...,...
1155,https://cards.scryfall.io/large/front/e/a/ea1a...,Zaffai__Thunder_Conductor_,2
1156,https://cards.scryfall.io/large/front/a/c/ac81...,Zagras__Thief_of_Heartbeats_,2
1157,https://cards.scryfall.io/large/front/8/8/880f...,Zahid__Djinn_of_the_Lamp_,2
1158,https://cards.scryfall.io/large/front/9/c/9c6d...,Zalto__Fire_Giant_Duke_,2
